#  Інтервальні методи для  систем рівнянь

In [25]:
!pip install -q pyinterval

In [26]:
from interval import interval
from interval import imath
import pandas as pd
import numpy as np
import math

In [27]:
def calculate_determinant(m):
    return m[0][0]*m[1][1] - m[0][1]*m[1][0]

def calculate_width(interv):
    return sum([x.sup - x.inf for x in interv])

def calculate_middle_point(interv):
    return (interv[0].sup + interv[0].inf) / 2

def to_string(interv):
    return f"[{interv[0].inf:9.7f}, {interv[0].sup:9.7f}]"

def half_interval(x):
    mid = calculate_middle_point(x)
    return (interval[x[0].inf, mid], interval[mid, x[0].sup])

def to_table(x, x_width, y, y_width, root, tbl):
    tbl.loc[len(tbl)] = [to_string(x), x_width, to_string(y), y_width, root]

In [28]:
func = lambda x, y: (x**2 - y, x**2 + y - x - 1) #x^2-y=0,x^2+y-x-1=0
dfunc = lambda x, y: ((2*x, -1), (2*x-1, 1))
initial_intervals = (interval[-0.75, 1.2], interval[0, 1.2])
e = 0.01

## Інтервальний метод Ньютона у формі Мура

In [29]:
def moore_system(f, df, intervals, e, tbl):
    x, y = intervals
    x_width = calculate_width(x)
    y_width = calculate_width(y)

    f_xy = f(x, y)

    if 0 not in f_xy[0] or 0 not in f_xy[1]:
        to_table(x, x_width, y, y_width, '', tbl)
        return

    if max(x_width, y_width) < e:
        to_table(x, x_width, y, y_width, 'Корінь', tbl)
        return

    x_middle = calculate_middle_point(x)
    y_middle = calculate_middle_point(y)
    df_xy = df(x, y)
    df_xy_det = interval(calculate_determinant(df_xy))

    if 0 in df_xy_det:
        x_left, x_right = half_interval(x)
        y_left, y_right = half_interval(y)
        if x_width < e:
            to_table(x, x_width, y, y_width, '', tbl)
            moore_system(f, df, (x, y_left), e, tbl)
            moore_system(f, df, (x, y_right), e, tbl)
        elif y_width < e:
            to_table(x, x_width, y, y_width, '', tbl)
            moore_system(f, df, (x_left, y), e, tbl)
            moore_system(f, df, (x_right, y), e, tbl)
        else:
            to_table(x, x_width, y, y_width, '', tbl)
            moore_system(f, df, (x_left, y_left), e, tbl)
            moore_system(f, df, (x_left, y_right), e, tbl)
            moore_system(f, df, (x_right, y_left), e, tbl)
            moore_system(f, df, (x_right, y_right), e, tbl)
        return

    f1m, f2m = f(x_middle, y_middle)
    df1x, df1y, df2x, df2y = *df_xy[0], *df_xy[1]
    N_x = x_middle + ((-df2y/df_xy_det)*f1m + (df1y/df_xy_det)*f2m)
    N_y = y_middle + ((df2x/df_xy_det)*f1m + (-df1x/df_xy_det)*f2m)
    x_next = N_x & x
    y_next = N_y & y

    if not x_next or not y_next:
        to_table(x, x_width, y, y_width, '', tbl)
        return

    to_table(x, x_width, y, y_width, '', tbl)
    moore_system(f, df, (x_next, y_next), e, tbl)

In [30]:
moore_system_result = pd.DataFrame(columns=['Інтервал Xi', 'Ширина Xi', 'Інтервал Yi', 'Ширина Yi', 'Корінь?'])
moore_system(func, dfunc, initial_intervals, e, moore_system_result)
moore_system_result

,Інтервал Xi,Ширина Xi,Інтервал Yi,Ширина Yi,Корінь?
0,"[-0.7500000, 1.2000000]",1.950000,"[0.0000000, 1.2000000]",1.200000,
1,"[-0.7500000, 0.2250000]",0.975000,"[0.0000000, 0.6000000]",0.600000,
2,"[-0.7500000, -0.4124219]",0.337578,"[0.0000000, 0.6000000]",0.600000,
3,"[-0.5170051, -0.4842851]",0.032720,"[0.1932610, 0.2905067]",0.097246,
4,"[-0.5001132, -0.4998868]",0.000226,"[0.2496605, 0.2503543]",0.000694,Корінь
5,"[-0.7500000, 0.2250000]",0.975000,"[0.6000000, 1.2000000]",0.600000,
6,"[0.2250000, 1.2000000]",0.975000,"[0.0000000, 0.6000000]",0.600000,
7,"[0.2250000, 0.7125000]",0.487500,"[0.0000000, 0.3000000]",0.300000,
8,"[0.2250000, 0.7125000]",0.487500,"[0.3000000, 0.6000000]",0.300000,
9,"[0.7125000, 1.2000000]",0.487500,"[0.0000000, 0.3000000]",0.300000,


## Інтервальний метод Ньютона у формі Хансена

In [31]:
def interval_calc(first, second):
    a = first[0].inf
    b = first[0].sup
    c = second[0].inf
    d = second[0].sup
    if b<=0 and d==0:
        return interval[b/c, inf]
    elif b<=0 and c<0 and d>0:
        return interval[-inf,b/d] | interval[b/c,inf]
    elif b<=0 and c==0:
        return interval[-inf,b/d]
    elif a<0 and b>0:
        return interval[-inf,inf]
    elif a>=0 and d==0:
        return interval[-inf,a/c]
    elif a>=0 and c<0 and d>0:
        return interval[-inf,a/c] | interval[a/d,inf]
    elif a>=0 and c==0:
        return interval[a/d, inf]
    else:
        return first/second

def hansen_system(f, df, intervals, e, tbl):
    x, y = intervals
    x_width = calculate_width(x)
    y_width = calculate_width(y)    

    f_xy = f(x, y)
    if 0 not in f_xy[0] or 0 not in f_xy[1]:
        to_table(x, x_width, y, y_width, '', tbl)
        return

    if max(x_width, y_width) < e:
        to_table(x, x_width, y, y_width, 'Корінь', tbl)
        return

    x_middle = calculate_middle_point(x)
    y_middle = calculate_middle_point(y)
    df_xy = df(x, y)
    df_xy_det = interval(calculate_determinant(df_xy))  
    f1m, f2m = f(x_middle, y_middle)

    if f1m == 0.0 and f2m == 0.0:
        to_table(x, x_width, y, y_width, '', tbl)
        x_left, x_right = half_interval(x)
        y_left, y_right = half_interval(y)
        hansen_system(f, df, (x_left, y_left), e, tbl)
        hansen_system(f, df, (x_left, y_right), e, tbl)
        hansen_system(f, df, (x_right, y_left), e, tbl)
        hansen_system(f, df, (x_right, y_right), e, tbl)
        return

    df1x, df1y, df2x, df2y = *df_xy[0], *df_xy[1]
    U_x = x_middle + ((-df2y/df_xy_det)*f1m + (df1y/df_xy_det)*f2m)
    U_y = y_middle + ((df2x/df_xy_det)*f1m + (-df1x/df_xy_det)*f2m)
    x_next = U_x & x
    y_next = U_y & y

    if not x_next or not y_next:
        to_table(x, x_width, y, y_width, '', tbl)
        return

    if x == x_next and y == y_next:
        if(x_width > y_width):
            to_table(x, x_width, y, y_width, '', tbl)
            x_left, x_right = half_interval(x)
            hansen_system(f, df, (x_left, y), e, tbl)
            hansen_system(f, df, (x_right, y), e, tbl)
        else:
            to_table(x, x_width, y, y_width, '', tbl)
            y_left, y_right = half_interval(y)
            hansen_system(f, df, (x, y_left), e, tbl)
            hansen_system(f, df, (x, y_right), e, tbl)
        return

    to_table(x, x_width, y, y_width, '', tbl)
    for xi in x_next:
        for yi in y_next:
            hansen_system(f, df, (interval(xi), interval(yi)), e, tbl)

In [32]:
hansen_system_result = pd.DataFrame(columns=['Інтервал Xi', 'Ширина Xi', 'Інтервал Yi', 'Ширина Yi', 'Корінь?'])
hansen_system(func, dfunc, initial_intervals, e, hansen_system_result)
hansen_system_result

,Інтервал Xi,Ширина Xi,Інтервал Yi,Ширина Yi,Корінь?
0,"[-0.7500000, 1.2000000]",1.950000,"[0.0000000, 1.2000000]",1.200000,
1,"[-0.7500000, 0.2250000]",0.975000,"[0.0000000, 1.2000000]",1.200000,
2,"[-0.7500000, -0.4124219]",0.337578,"[0.0000000, 1.2000000]",1.200000,
3,"[-0.5504094, -0.4508808]",0.099529,"[0.0588002, 0.3733573]",0.314557,
4,"[-0.5014877, -0.4985072]",0.002980,"[0.2458019, 0.2547939]",0.008992,Корінь
5,"[0.2250000, 1.2000000]",0.975000,"[0.0000000, 1.2000000]",1.200000,
6,"[0.2250000, 1.2000000]",0.975000,"[0.0000000, 0.6000000]",0.600000,
7,"[0.2250000, 0.7125000]",0.487500,"[0.0000000, 0.6000000]",0.600000,
8,"[0.7125000, 1.2000000]",0.487500,"[0.0000000, 0.6000000]",0.600000,
9,"[0.2250000, 1.2000000]",0.975000,"[0.6000000, 1.2000000]",0.600000,


## Інтервальний метод Ньютона у формі  Кравчика

In [33]:
def krawczyk_system(f, df, intervals, e, tbl):
    x, y = intervals
    x_width = calculate_width(x)
    y_width = calculate_width(y)

    f_xy = f(x, y)
    if 0 not in f_xy[0] or 0 not in f_xy[1]:
        to_table(x, x_width, y, y_width, '', tbl)
        return

    if max(x_width, y_width) < e:
        to_table(x, x_width, y, y_width, 'Корінь', tbl)
        return

    x_middle = calculate_middle_point(x)
    y_middle = calculate_middle_point(y)
    df_xy = df(x, y)
    df_xy_det = interval(calculate_determinant(df_xy))

    if 0 in df_xy_det:
        x_left, x_right = half_interval(x)
        y_left, y_rigth = half_interval(y)
        if x_width < e:
            to_table(x, x_width, y, y_width, '', tbl)
            krawczyk_system(f, df, (x, y_left), e, tbl)
            krawczyk_system(f, df, (x, y_right), e, tbl)
        elif y_width < e:
            to_table(x, x_width, y, y_width, '', tbl)
            krawczyk_system(f, df, (x_left, y), e, tbl)
            krawczyk_system(f, df, (x_right, y), e, tbl)
        else:
            to_table(x, x_width, y, y_width, '', tbl)
            krawczyk_system(f, df, (x_left, y_left), e, tbl)
            krawczyk_system(f, df, (x_left, y_rigth), e, tbl)
            krawczyk_system(f, df, (x_right, y_left), e, tbl)
            krawczyk_system(f, df, (x_right, y_rigth), e, tbl)
        return

    f1m, f2m = f(x_middle, y_middle)
    df_mid = df(x_middle, y_middle)
    df1x,  df1y,  df2x,  df2y =  *df_xy[0], *df_xy[1]
    df1xm, df1ym, df2xm, df2ym = *df_mid[0], *df_mid[1]
    det = calculate_determinant(df_mid)
    retard_x = (1 + df2x*df1ym/det - df1x*df2ym/det)*(x-x_middle) \
                + ( df2y*df1ym/det - df1y*df2ym/det)*(y-y_middle)
    retard_y = (     - df2x*df1xm/det + df1x*df2xm/det)*(x-x_middle) \
                + (1 - df2y*df1xm/det + df1y*df2xm/det)*(y-y_middle)
    K_x = x_middle + ((-df2ym/det)*f1m + ( df1ym/det)*f2m ) + retard_x
    K_y = y_middle + (( df2xm/det)*f1m + (-df1xm/det)*f2m ) + retard_y
    x_next = K_x & x
    y_next = K_y & y

    if not x_next or not y_next:
        to_table(x, x_width, y, y_width, '', tbl)
        return

    to_table(x, x_width, y, y_width, '', tbl)
    krawczyk_system(f, df, (x_next, y_next), e, tbl)

In [34]:
krawczyk_system_result = pd.DataFrame(columns=['Інтервал Xi', 'Ширина Xi', 'Інтервал Yi', 'Ширина Yi', 'Корінь?'])
krawczyk_system(func, dfunc, initial_intervals, e, krawczyk_system_result)
krawczyk_system_result

,Інтервал Xi,Ширина Xi,Інтервал Yi,Ширина Yi,Корінь?
0,"[-0.7500000, 1.2000000]",1.950000,"[0.0000000, 1.2000000]",1.200000,
1,"[-0.7500000, 0.2250000]",0.975000,"[0.0000000, 0.6000000]",0.600000,
2,"[-0.7500000, -0.0913110]",0.658689,"[0.0000000, 0.6000000]",0.600000,
3,"[-0.6664276, -0.3429596]",0.323468,"[0.0307176, 0.4645888]",0.433871,
4,"[-0.5346749, -0.4653543]",0.069321,"[0.1976769, 0.3023085]",0.104632,
5,"[-0.5016017, -0.4983983]",0.003203,"[0.2475973, 0.2524027]",0.004805,Корінь
6,"[-0.7500000, 0.2250000]",0.975000,"[0.6000000, 1.2000000]",0.600000,
7,"[0.2250000, 1.2000000]",0.975000,"[0.0000000, 0.6000000]",0.600000,
8,"[0.2250000, 0.7125000]",0.487500,"[0.0000000, 0.3000000]",0.300000,
9,"[0.2250000, 0.7125000]",0.487500,"[0.3000000, 0.6000000]",0.300000,
